In [1]:
#filename,random_list,totalinfo,datakeylist,누적분포,시행수
class 데이터처리:
  random_list=[]
  filename="example"
  datakeylist=[]
  전체정보리스트=[]
  전체누적분포=[]
  t_sum=0

  def __init__(self):
    pass

  def 랜덤함수(self,_a,_b,_k): #_a~_b범위에서 중복포함 랜덤으로 _k개 뽑아 리스트로 돌려주는 함수
    import random
    _list=[]
    while len(_list) < _k:
      _list.append(random.randint(_a,_b)) 
    return _list

  def 줄별저장(self,_file,_list):
    i=0
    for x in _list:
      if i<(len(_list)-1):
        _file.write(str(x)+",")
        i+=1
      else:
        _file.write(str(x)+"\n")

  #_lists=[[],[],[],[]] 꼴인 2차원 배열로 가정
  def csv저장(self,_name,_lists):
    _f=open("{}.csv".format(_name), mode='w', newline="", encoding='utf-8-sig')
    for _row in _lists:
      self.줄별저장(_f,_row)
    _f.close

  def csv저장(self):
    _f=open("{}.csv".format(self.filename), mode='w', newline="", encoding='utf-8-sig')
    for _row in self.random_list:
      self.줄별저장(_f,_row)
    _f.close

    #정수화 함수 #스트링으로 된값을 정수화시킴
  def 정수화 (self,_row):
    for i in range(len(_row)):
      _row[i]=int(_row[i])
    return _row

  #각행 평균
  def 각행평균(self,_row):
    _sum=0
    for x in _row:
      x=int(x)
      _sum += x
    _average = _sum/len(_row)
    return _average

  #각행 편차 : 편차는 리스트로 만들어두지만 최종 리턴값은 표준편차
  def 각행편차(self,_row):
    _average= self.각행평균(_row)
    편차리스트=[]
    for x in _row:
      x=int(x)
      _deviation = x-_average
      편차리스트.append(_deviation)
    #print(편차리스트) #정상작동 확인용

    _dsum=0
    for y in 편차리스트:
      _dsum += y*y
    _variance = _dsum/len(_row)
    _sd = _variance **0.5
    return _sd
  #계산기로 도출한 값과 같음

  #행별누적분포, 각 요소가 몇번 반복됐는지 누적해서 저장
  #누적분포는 dictionary로 저장해 정수값:누적분포로 관리하는게 이상적일듯
  def 행별누적분포(self,_row):
    _행별누적분포dic=dict.fromkeys(_row)
    for x in _row:
      _행별누적분포dic[x]=_row.count(x)
    return(_행별누적분포dic)

  #난수 리스트를 입력받은 값을 바탕으로 여러개 생성하고 이를 반환하는 함수
  def 난수리스트입력받아생성(self):
    import random
    #난수 리스트 생성후 저장
    _random_list=[]
    while True:
      print("범위 지정할 숫자 2개, 랜덤한 숫자 생성 갯수를 " "로 구분해 a b n꼴로 입력해주세요.")
      abn=input("입력을 멈추시려면 n을 입력해주세요.")
      if abn=="n":
        _filename=input("저장할 파일 이름을 입력해주세요.")
        self.filename=_filename
        break

      n=0
      _tlist=[]
      for i in range(len(abn)):
        if abn[i]==" ":
          _tlist.append(abn[n:i])
          n=i+1
        if i == len(abn)-1:
          _tlist.append(abn[n:])
      
      self.정수화(_tlist)
      a=_tlist[0]; b=_tlist[1]; n=_tlist[2]
      if(a>b):
        max=a
        min=b
      else:
        max=b
        min=a
      _random_list.append(self.랜덤함수(int(a),int(b),int(n)))
    #입력 제대로 안하고 끝냈을때 작동용 랜덤함수 리스트 생성
    if len(_random_list)==0:
      _random_list=(self.랜덤함수(1,random.randint(1,30),random.randint(1,20)),self.랜덤함수(1,random.randint(1,30),random.randint(1,20)),self.랜덤함수(1,random.randint(1,30),random.randint(1,20)),self.랜덤함수(1,random.randint(1,30),random.randint(1,20)),self.랜덤함수(1,random.randint(1,30),random.randint(1,20)))
    self.random_list=_random_list
    return _filename, _random_list

  def csv파일리스트화(self):
    #저장된 파일 읽어서 처리해 리스트로 저장
    import csv
    f=open("{}.csv".format(self.filename),mode='r',encoding='utf-8-sig')
    rd=csv.reader(f)
    전체정보리스트=[]
    for row in rd:
      self.정수화(row)
      행별정보리스트=[self.각행평균(row),self.각행편차(row),self.행별누적분포(row)]
      전체정보리스트.append(행별정보리스트)

    #print(전체정보리스트) #정상작동확인용
    self.전체정보리스트=전체정보리스트
    return 전체정보리스트
    f.close()

  def 데이터키값추출(self):
    #전체정보리스트[i][2] 는 i번째 열의 누적분포
    datakeylist=[] #누적분포dic에서 key값을 모아낼 리스트
    for i in range(len(self.전체정보리스트)):
      _tempkeys=self.전체정보리스트[i][2].keys() #각행 누적분포에서 키값만 뽑아냄
      datakeylist.extend(_tempkeys) #일단 통째로 넣음

    datakeylist = set(datakeylist)
    datakeylist = list(datakeylist) #set으로 바꿨다 list로 다시바꾸는거를 통해 편하게 중복제거
    datakeylist.sort()

    #print(datakeylist) #정상작동 확인용
    self.datakeylist=datakeylist
    return datakeylist
  
  def 전체누적분포구하기(self):
    #datakeylist바탕으로 전체 누적분포 구하기
    #dic=dict.fromkeys(seq[,v]) : 시퀀스의 값을 키로 갖고 그 값을 v로 갖는 새로운 dictionary 생성 (v는 보통 생략됨 생략하면 None값 가짐)
    전체누적분포=dict.fromkeys(self.datakeylist,0) #초기값 0
    t_sum=0 #발생확률 구하기위해 필요
    for x in self.datakeylist:
      d_sum=0
      for i in range(len(self.전체정보리스트)):
        _t=self.전체정보리스트[i][2]
        if _t.get(x)!=None:
          d_sum += self.전체정보리스트[i][2][x]
          t_sum += self.전체정보리스트[i][2][x]
      전체누적분포[x]=d_sum

    #print(전체누적분포) #정상작동확인용
    self.전체누적분포=전체누적분포
    self.t_sum=t_sum
    return 전체누적분포, self.t_sum

  def 라벨저장(self):
    #가공한 리스트로 형식에 맞춰 새로운 파일 저장
    #위에서 구한 전체정보리스트와 datakeylist 바탕 각행별 누적분포 파일로 저장
    f=open("{}의 결과물.csv".format(self.filename), mode='w',encoding='utf-8-sig')
    #첫번째 행에 전체정보리스트에 있는 데이터들과 평균, 표준편차 라벨 입력
    f.write("구분,")
    datakeylist=self.datakeylist
    for x in datakeylist:
      f.write("data: "+str(x)+',')
    f.write("평균,표준편차\n")

  def 행별입력(self):
    #각 행별로 데이터 입력
    f=open("{}의 결과물.csv".format(self.filename), mode='a',encoding='utf-8-sig')
    datakeylist=self.datakeylist
    전체정보리스트=self.전체정보리스트
    for i in range(len(전체정보리스트)):
      f.write("{}행,".format(i+1))
      for x in datakeylist:
        #dic.get(key[,d])
        _t=전체정보리스트[i][2]
        if _t.get(x)!=None:
          f.write(str(_t[x])+',')
        else:
          f.write(str(0)+',')
      f.write(str(전체정보리스트[i][0])+','+str(전체정보리스트[i][1])+'\n')

  def 마무리입력(self):
    #마지막에 전체누적분포와 발생확률 입력
    #전체 누적분포
    f=open("{}의 결과물.csv".format(self.filename), mode='a',encoding='utf-8-sig')
    f.write("전체 누적분포,")
    for x in self.datakeylist:
      f.write(str(self.전체누적분포[x])+',')
    f.write("총 시행수,{}".format(str(self.t_sum)))
    f.write('\n')
    #발생확률
    f.write("발생확률,")
    for x in self.datakeylist:
      f.write(str(self.전체누적분포[x]/self.t_sum)+',')
    f.write('\n')

In [2]:
dp=데이터처리()
dp.난수리스트입력받아생성()
dp.csv저장()
dp.csv파일리스트화()
dp.데이터키값추출()
dp.전체누적분포구하기()
dp.라벨저장()
dp.행별입력()
dp.마무리입력()

범위 지정할 숫자 2개, 랜덤한 숫자 생성 갯수를 로 구분해 a b n꼴로 입력해주세요.
입력을 멈추시려면 n을 입력해주세요.n
저장할 파일 이름을 입력해주세요.test


In [ ]:
#csv파일에서 i행 j열에 접근
#csv파일 리스트화
def csvTolist(filename):
  import csv
  f= open(filename,mode="r",encoding='utf-8-sig')
  rd=csv.reader(f)
  csvlist=[]
  for line in rd:
    csvlist.append(line)
  return csvlist

#csvlist[i][j]로 i행 j열에 손쉽게 접근 가능